In [3]:
# Ref
# - https://numpy.org/doc/stable/reference/maskedarray.generic.html
# - https://joehamman.com/2013/10/12/plotting-netCDF-data-with-Python/
# - http://www.sidc.be/silso/infosnmtot

In [4]:
from netCDF4 import Dataset
from datetime import datetime, timedelta
import numpy as np
import pandas as pd

# Sunspot Data Prep

In [74]:
df = pd.read_csv('SN_m_tot_V2.0.csv', delimiter=";", header=None)
df.columns = ['Year', 'Month', 'Date in fraction of year', 'Monthly mean total sunspot number', 'Monthly mean standard deviation', 'Number of observations', 'Definitive/provisional marker']
df = df[['Year', 'Month', 'Monthly mean total sunspot number']]
df['Date'] = pd.to_datetime(df[['Year', 'Month']].assign(DAY=1))
df.set_index('Date', inplace=True)
df = df[['Monthly mean total sunspot number']]
df.columns = ['Sunspot']
df.to_csv('Monthwise_Sunspot.csv')

# Precipitation Data Prep

In [91]:
file_name = "raw_datasets/final_kelvin.nc"
file = Dataset(file_name, mode='r')

In [95]:
file

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): lat(124), lon(147), time(24106)
    variables(dimensions): float64 lat(lat), float64 lon(lon), int64 time(time), float64 prcp(lat,lon,time), float64 tasmax(lat,lon,time), float64 tasmin(lat,lon,time)
    groups: 

In [96]:
location_wise = np.zeros(file['time'].shape)
count = 0
for lat in file['prcp']:
    count+=1
    if count%10 == 0:
        print(count)
        
    for long in lat:
        long = long[~long.mask]
        if long != []:
            location_wise = np.vstack([location_wise, long])

In [ ]:
pd.DataFrame(a).to_csv('daywise_precipitation/Latitude_Longitude_Wise_Sequence_of_Days.csv', index=False)

In [5]:
location_wise = pd.read_csv('daywise_precipitation/Latitude_Longitude_Wise_Sequence_of_Days.csv')
location_wise = location_wise.to_numpy()
location_wise[location_wise == 0] = np.nan
location_wise_mean = np.nanmean(location_wise, axis=0)
np.save('daywise_precipitation/Daywise_Precipitation', location_wise_mean) # List of precip values on each day starting 1951-01-02

In [58]:
precipitaion = np.load('daywise_precipitation/Daywise_Precipitation.npy')

list_dates = []
for i in range(24106):
    list_dates.append(datetime.strptime('1951-01-02',  '%Y-%m-%d') + timedelta(i))
    
df_date_precip = pd.concat([pd.DataFrame(list_dates), pd.DataFrame(precipitaion)],axis=1)
df_date_precip.columns = ['Date', 'Precipitation']
df_date_precip.set_index('Date', inplace=True)
df_date_precip = df_date_precip.resample('M').mean()
df_date_precip.to_csv('monthwise_datasets/Monthwise_Precipitation.csv') # Monthwise mean of precipitation value